In [1]:
# 🔧 STEP 1: Install Required Libraries
!pip install fastapi pyngrok uvicorn PyPDF2 torch networkx matplotlib nest_asyncio pydantic sentence_transformers

In [11]:
# 🔧 STEP 2: Import modules and set up FastAPI app
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse
from pydantic import BaseModel
import PyPDF2
import io
import torch
import torch.nn as nn
import networkx as nx
import matplotlib.pyplot as plt
from pyngrok import ngrok
import nest_asyncio
import uvicorn

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "🎉 FastAPI is running! Use /upload-pdf, /summarize, /ask, or /mindmap endpoints."}

@app.get("/summarize/")
def summarize():
    with open("lecture.txt", "r") as f:
        text = f.read()

    sentences = text.split(". ")
    scored = sorted(sentences, key=lambda s: len(s), reverse=True)
    top = scored[:5]  # top 5 longest, basic summarization logic
    summary = ". ".join(top)
    return {"summary": summary}



In [3]:
# 🔧 STEP 3: Define a minimal custom Transformer model
class SimpleTransformer(nn.Module):
    def _init_(self, vocab_size=5000, d_model=64, nhead=4, nlayers=2, dim_feed=128):
        super()._init_()
        self.embed = nn.Embedding(vocab_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feed)
        self.encoder = nn.TransformerEncoder(encoder_layer, nlayers)
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feed)
        self.decoder = nn.TransformerDecoder(decoder_layer, nlayers)
        self.output = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt):
        src = self.embed(src)
        tgt = self.embed(tgt)
        memory = self.encoder(src)
        out = self.decoder(tgt, memory)
        return self.output(out)

model = SimpleTransformer()

In [4]:
# 🔧 STEP 4: Handle PDF upload and extract text
@app.post("/upload-pdf/")
async def upload_pdf(file: UploadFile = File(...)):
    contents = await file.read()
    reader = PyPDF2.PdfReader(io.BytesIO(contents))
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    with open("lecture.txt", "w") as f:
        f.write(text)
    return {"message": "PDF uploaded and text extracted successfully!"}

In [5]:
# 🔧 STEP 5: Basic summarizer using scoring
@app.get("/summarize/")
def summarize():
    with open("lecture.txt", "r") as f:
        text = f.read()

    sentences = text.split(". ")
    scored = sorted(sentences, key=lambda s: len(s), reverse=True)
    top = scored[:5]  # top 5 longest, basic summarization logic
    summary = ". ".join(top)
    return {"summary": summary}

In [6]:
# 🔧 STEP 6: Simple Q&A using keyword match
class Question(BaseModel):
    question: str

@app.post("/ask/")
def ask_qn(q: Question):
    with open("lecture.txt", "r") as f:
        text = f.read().lower()
    question = q.question.lower()
    words = question.split()
    best_line = ""
    max_overlap = 0
    for line in text.split("."):
        overlap = len(set(words) & set(line.strip().split()))
        if overlap > max_overlap:
            best_line = line.strip()
            max_overlap = overlap
    return {"answer": best_line if best_line else "Sorry, I couldn’t find a relevant answer."}

In [7]:
# 🔧 STEP 7: Generate a basic mind map using keyword graph
@app.get("/mindmap/")
def mind_map():
    with open("lecture.txt", "r") as f:
        text = f.read().lower()

    # Extract keywords (simple version)
    keywords = [word for word in text.split() if len(word) > 5]
    freq = {}
    for word in keywords:
        freq[word] = freq.get(word, 0) + 1
    sorted_words = sorted(freq.items(), key=lambda x: x[1], reverse=True)[:10]

    # Create mind map graph
    G = nx.Graph()
    central = "Lecture"
    G.add_node(central)
    for word, _ in sorted_words:
        G.add_node(word)
        G.add_edge(central, word)

    # Draw mind map
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=2000, font_size=12)
    plt.savefig("mindmap.png")

    return FileResponse("mindmap.png", media_type="image/png")

In [12]:
!ngrok config add-authtoken 2ydPoK24vSLZcscOVhhn2XHCQt3_7owGMrLRSrDZhXt8k23Y3


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
import nest_asyncio
import threading
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("🚀 Your public FastAPI URL:", public_url)

def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=start_server)
thread.start()


🚀 Your public FastAPI URL: NgrokTunnel: "https://203a-34-106-244-226.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [10560]
INFO:     Waiting for application startup.


In [ ]:
# 📌 Step 1: Install necessary libraries
!pip install streamlit pyngrok -q

# 📌 Step 2: Import libraries
from pyngrok import ngrok
import streamlit as st
import requests

# ✅ FIXED: Connect ngrok using the correct format
public_url = ngrok.connect("http://localhost:8501")
print("🔗 Streamlit frontend URL:", public_url)

# 🔗 Your backend URL remains the same
BASE_URL = "https://203a-34-106-244-226.ngrok-free.app"

# 📄 Step 3: Define the frontend app code
code = f'''
import streamlit as st
import requests

st.set_page_config(page_title="📚 Classroom Summarizer")

st.title("📚 Classroom Lecture Summarizer Bot")

uploaded_file = st.file_uploader("Upload a lecture PDF", type=["pdf"])
if st.button("Upload PDF") and uploaded_file:
    try:
        res = requests.post("{BASE_URL}/upload-pdf/", files={{"file": uploaded_file}})
        st.success("✅ PDF uploaded successfully!")
    except Exception as e:
        st.error("❌ Upload failed: " + str(e))

if st.button("📄 Get Summary"):
    try:
        res = requests.get("{BASE_URL}/summarize/")
        st.info(res.json()["summary"])
    except:
        st.error("❌ Failed to fetch summary.")

st.subheader("❓ Ask a Question")
question = st.text_input("Type your question here")
if st.button("Ask"):
    try:
        res = requests.post("{BASE_URL}/ask/", json={{"question": question}})
        st.success(res.json()["answer"])
    except:
        st.error("❌ Failed to get answer.")

if st.button("🧠 Get Mind Map"):
    try:
        res = requests.get("{BASE_URL}/mindmap/", stream=True)
        if res.status_code == 200:
            st.image(res.content)
        else:
            st.error("❌ Could not fetch mind map.")
    except:
        st.error("❌ Mind map fetch failed.")
'''

# 📌 Step 4: Save the code to app.py
with open("app.py", "w") as f:
    f.write(code)

# 📌 Step 5: Run the Streamlit app
!streamlit run app.py &


🔗 Streamlit frontend URL: NgrokTunnel: "https://e9fe-34-106-244-226.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.244.226:8501

INFO:     34.106.244.226:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK
INFO:     34.106.244.226:0 - "GET /summarize/ HTTP/1.1" 200 OK
INFO:     34.106.244.226:0 - "GET /mindmap/ HTTP/1.1" 200 OK
INFO:     34.106.244.226:0 - "GET /summarize/ HTTP/1.1" 200 OK
INFO:     34.106.244.226:0 - "GET /mindmap/ HTTP/1.1" 200 OK
INFO:     34.106.244.226:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK
INFO:     34.106.244.226:0 - "GET /summarize/ HTTP/1.1" 200 OK
INFO:     34.106.244.226:0 - "GET /mindmap/ HTTP/1.1" 200 OK
